### Lendo imagem

In [ ]:
import SimpleITK as sitk
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


#root_dir linux
#root_dir ="/home/adriano/projeto_mestrado/modules"

#root_dir windows
root_dir = Path(r"C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari")
dir_images = f'/imagens/3d/'

image_path = f'{root_dir}{dir_images}I7025.nii.gz'
image = sitk.ReadImage(image_path) 

    
# Converter para numpy array
image_array = sitk.GetArrayFromImage(image)

# Exibir um slice no meio do volume
slice_idx = image_array.shape[0] // 2  # Pega o índice do slice central
slice_image = image_array[slice_idx, :, :]

# Plotar o slice
plt.figure(figsize=(8, 8))
plt.imshow(slice_image, cmap="gray")
plt.title(f"Slice {slice_idx}")
plt.axis("off")
plt.show()



### Função

In [6]:
import os
def read_directories(directory, img=None):
    # Get a list of filenames in the specified directory
    filenames = []
    for filename in os.listdir(directory):
        if img is not None:
            # If 'img' is provided, filter filenames containing it
            if img in filename:   
                filenames.append(filename)          
        else:
            filenames.append(filename)    
    return filenames

### Diretório das imagens

In [6]:
dir_images = f'work1/clinical_images'

### Filtro Anisotrópico

In [15]:

from pathlib import Path

import SimpleITK as sitk

def apply_anisotropic_diffusion_sitk(image_path, time_step, conductance, iterations, output_path):
    """
    Apply a 3D anisotropic diffusion filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - time_step (float): The time step for the diffusion process.
    - conductance (float): The conductance parameter for the diffusion process.
    - iterations (int): Number of iterations for the diffusion process.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Cast the image to a supported pixel type (float32)
    image = sitk.Cast(image, sitk.sitkFloat32)
    
    # Apply Anisotropic Diffusion filter
    anisotropic_diffusion_filter = sitk.CurvatureAnisotropicDiffusionImageFilter()
    anisotropic_diffusion_filter.SetTimeStep(time_step)
    anisotropic_diffusion_filter.SetConductanceParameter(conductance)
    anisotropic_diffusion_filter.SetNumberOfIterations(iterations)
    filtered_image = anisotropic_diffusion_filter.Execute(image)
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")

# Example usage
if __name__ == "__main__":
    array_images = read_directories(dir_images)    
    dir_output = f'work1/denoising/anisotropicDiffusion'  
    time_step, conductance, iterations =  0.0625, 1.0, 10
    for image in array_images:    
        output_name = f'{image}'
        apply_anisotropic_diffusion_sitk(f'{dir_images}/{image}', time_step, conductance, iterations, f'{dir_output}/{output_name}')
        

Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI002-Guys-0828-T1.nii.gz
Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI012-HH-1211-T1.nii.gz
Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI013-HH-1212-T1.nii.gz
Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI014-HH-1236-T1.nii.gz
Filtered image using SimpleITK saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/anisotropicDiffusion/_andif_0__IXI015-HH-1258-T1.nii.gz


### NLM - Skimage

In [14]:
# Need to install
# pip install scikit-image
# pip install imageio-ffmpeg (not sure if this is needed ???)
# pip install nibabel
# pip install PyWavelets

import numpy as np
from skimage.restoration import denoise_nl_means, estimate_sigma
import nibabel as nib
from pathlib import Path

def apply_nonlocal_means_skimage(image_path, patch_size, patch_distance, h, fast_mode, output_path):
    """
    Apply a 3D Non-Local Means filter to a 3D image using scikit-image.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - patch_size (int): The size of the patches used for denoising.
    - patch_distance (int): The distance for the neighborhood search.
    - h (float): The filtering parameter.
    - fast_mode (bool): Whether to use the fast mode or not.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = nib.load(image_path)
    image_data = image.get_fdata()
    
    # Estimate the noise standard deviation from the noisy image
    sigma_est = np.mean(estimate_sigma(image_data))
    
    # Apply Non-Local Means filter
    denoised_image = denoise_nl_means(image_data, patch_size=patch_size, patch_distance=patch_distance, h=h*sigma_est, fast_mode=fast_mode)
    
    # Save the filtered image
    denoised_image_nib = nib.Nifti1Image(denoised_image, image.affine)
    nib.save(denoised_image_nib, output_path)
    print(f"Filtered image using scikit-image saved to {output_path}")

# Example usage
if __name__ == "__main__":
    array_images = read_directories(dir_images)    
    dir_output = f'work1/denoising/non_local_means'
    i = 0
    patch_size, patch_distance, h = 3, 5, 1.15
    for image in array_images:    
        output_name = f'{image}'
        apply_nonlocal_means_skimage(f'{dir_images}/{image}',  patch_size, patch_distance, h, True, f'{dir_output}/{output_name}')


Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI002-Guys-0828-T1.nii.gz
Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI012-HH-1211-T1.nii.gz
Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI013-HH-1212-T1.nii.gz
Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI014-HH-1236-T1.nii.gz
Filtered image using scikit-image saved to C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari/images\work1\denoising/non_local_means/_nlm_0__IXI015-HH-1258-T1.nii.gz


### NLM SimpleITK

In [ ]:
import SimpleITK as sitk
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt



array_images = read_directories(dir_images)
for image in array_images:
    patch_radius = 4
   
    image_path = f'{dir_images}/{image}'
    print(image_path)
    image_read = sitk.ReadImage(image_path)
    # Apply Non-Local Means filter
    nlm_filter = sitk.PatchBasedDenoisingImageFilter()
    nlm_filter.SetPatchRadius(patch_radius)
    nlm_filter.SetNoiseModel(sitk.PatchBasedDenoisingImageFilter.GAUSSIAN)
    filtered_image = nlm_filter.Execute(image_read)
    output_path = f'{dir_images}/denoising/non_local_means/'
    output_name = f'{image}'
    # Save the filtered image
    sitk.WriteImage(filtered_image, f'{output_path}/{output_name}')
  

### Bilateral Filter

In [ ]:
import SimpleITK as sitk

def apply_bilateral_filter_sitk(image_path, domain_sigma, range_sigma, output_path):
    """
    Apply a 3D bilateral filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - domain_sigma (float): The standard deviation for the spatial Gaussian kernel.
    - range_sigma (float): The standard deviation for the range Gaussian kernel.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Apply Bilateral filter
    bilateral_filter = sitk.BilateralImageFilter()
    bilateral_filter.SetDomainSigma(domain_sigma)
    bilateral_filter.SetRangeSigma(range_sigma)
    filtered_image = bilateral_filter.Execute(image)
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")

if __name__ == "__main__":
    array_images = read_directories(dir_images)
    #root_dir = Path(r"C:\Users\adria\Documents\Doutorado\PDI\Notebooks - PDI\PDI_Ferrari")
    dir_output = f'work1/denoising/bilateral'
    domain_sigma, range_sigma =  2.0, 50.0   

for image in array_images:    
    output_name = f't{image}'
    apply_bilateral_filter_sitk(f'{dir_images}/{image}', domain_sigma, range_sigma, f'{dir_output}/{output_name}')
    


### G - Images

#### bilateral

In [22]:
dir_anisotropic = f'work1/denoising/bilateral/images_filtered'
dir_out_anisotropic = f'work1/denoising/bilateral/G'

for image in array_images:
    original_image = sitk.ReadImage(f'{dir_images}/{image}')
    filtered_image = sitk.ReadImage(f'{dir_anisotropic}/{image}')
    image_array = sitk.GetArrayFromImage(original_image)
    image_array_filtered = sitk.GetArrayFromImage(filtered_image)
    image_g = abs(image_array-image_array_filtered)
    filtered_image = sitk.GetImageFromArray(image_g)
    sitk.WriteImage(filtered_image, f'{dir_out_anisotropic}/{image}')


#### anisotropicDiffusion

In [23]:
dir_anisotropic = f'work1/denoising/anisotropicDiffusion/images_filtered'
dir_out_anisotropic = f'work1/denoising/anisotropicDiffusion/G'

for image in array_images:
    original_image = sitk.ReadImage(f'{dir_images}/{image}')
    filtered_image = sitk.ReadImage(f'{dir_anisotropic}/{image}')
    image_array = sitk.GetArrayFromImage(original_image)
    image_array_filtered = sitk.GetArrayFromImage(filtered_image)
    image_g = abs(image_array-image_array_filtered)
    filtered_image = sitk.GetImageFromArray(image_g)
    sitk.WriteImage(filtered_image, f'{dir_out_anisotropic}/{image}')


#### non_local_means

In [25]:
dir_anisotropic = f'work1/denoising/non_local_means/images_filtered'
dir_out_anisotropic = f'work1/denoising/non_local_means/G'

for image in array_images:
    original_image = sitk.ReadImage(f'{dir_images}/{image}')
    filtered_image = sitk.ReadImage(f'{dir_anisotropic}/{image}')
    image_array = sitk.GetArrayFromImage(original_image)
    image_array_filtered = sitk.GetArrayFromImage(filtered_image)
    image_g = abs(image_array-image_array_filtered)
    filtered_image = sitk.GetImageFromArray(image_g)
    sitk.WriteImage(filtered_image, f'{dir_out_anisotropic}/{image}')


### Cálculo dos volumes

In [ ]:
import nibabel as nib
import numpy as np

import pandas as pd

def criar_planilha(resultados, arquivo_saida="volumes.xlsx"):
    """
    Cria uma planilha a partir dos resultados de volumes de WM, GM e CSF.

    Parâmetros:
    - resultados: Lista de dicionários com os resultados.
      Cada dicionário deve conter: 'Código da Imagem', 'WM_volume_mm3', 'GM_volume_mm3', 'CSF_volume_mm3'.
    - arquivo_saida: Nome do arquivo de saída (com extensão .xlsx).
    """
    # Criar DataFrame com os dados
    df = pd.DataFrame(resultados)

    # Salvar DataFrame como arquivo Excel
    df.to_excel(arquivo_saida, index=False)
    print(f"Planilha salva como: {arquivo_saida}")

def calcular_volumes(img):
    """
    Calcula os volumes de WM, GM e CSF em mm³ a partir de uma imagem segmentada,
    verificando as dimensões do voxel automaticamente.
    
    Parâmetros:
    - image_path: Caminho para o arquivo NIfTI da imagem segmentada.
    
    Retorna:
    - Um dicionário com os volumes de WM, GM e CSF em mm³.
    """
    # Carregar a imagem segmentada
   
    data = img.get_fdata()
    print(f'data: {data}')
    
    # Obter dimensões do voxel a partir do cabeçalho da imagem
    voxel_dims = img.header.get_zooms()[:3]  # Pega os 3 primeiros valores (x, y, z)
    voxel_volume = np.prod(voxel_dims)  # Calcula o volume do voxel em mm³
    print(f"voxel_dims:{voxel_dims}")
    print(f"voxel_volume:{voxel_volume}")
    # Identificar classes (0: fundo, 1: WM, 2: GM, 3: CSF)
    wm_volume = np.sum(data == 1) * voxel_volume
    gm_volume = np.sum(data == 2) * voxel_volume
    csf_volume = np.sum(data == 3) * voxel_volume
    
    # Retornar os volumes
    return wm_volume, gm_volume, csf_volume
      

# Exemplo de uso
if __name__ == "__main__":
    # Caminho para o arquivo segmentado (formato NIfTI .nii ou .nii.gz)
    #image_path = "work1/denoising/non_local_means/images_filtered/IXI002-Guys-0828-T1.nii.gz"
    dir_images_filtered = 'work1/denoising/non_local_means/images_filtered'
    dir_teste = 'work1/clinical_images'
    array_images_filtered = read_directories(dir_images_filtered)
    #array_images_filtered = read_directories(dir_teste)
    array_dict = []
    for image in array_images_filtered:
        img = nib.load(f'{dir_images_filtered}/{image}')
      
        wm_volume, gm_volume, csf_volume = calcular_volumes(img)
        dict ={
            "imagem": image,
            "wm":wm_volume,
            "gm":gm_volume,
            "csf":csf_volume
        }
        print(dict)
        array_dict.append(dict)
    
    criar_planilha(array_dict)

In [28]:
for image in array_images_filtered:
     img = nib.load(f'{dir_images_filtered}/{image}')
     print(img)
{'imagem': 'IXI002-Guys-0828-T1.nii.gz', 'wm': np.float64(269.99932861328125), 'gm': np.float64(439.80359387397766), 'csf': np.float64(589.5688464641571)}


<class 'nibabel.nifti1.Nifti1Image'>
data shape (256, 256, 150)
affine:
[[ 0.00000000e+00  0.00000000e+00  1.19999695e+00 -8.86398926e+01]
 [-9.30352330e-01  1.15545668e-01  0.00000000e+00  1.16532005e+02]
 [ 1.15545668e-01  9.30352330e-01 -2.49799545e-16 -1.12113556e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : np.bytes_(b'')
db_name         : np.bytes_(b'')
extents         : 0
session_error   : 0
regular         : np.bytes_(b'')
dim_info        : 0
dim             : [  3 256 256 150   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [-1.        0.9375    0.9375    1.199997  1.        1.        1.
  1.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unk

{'imagem': 'IXI002-Guys-0828-T1.nii.gz',
 'wm': np.float64(269.99932861328125),
 'gm': np.float64(439.80359387397766),
 'csf': np.float64(589.5688464641571)}

In [27]:
import nibabel as nib
import numpy as np

from scipy.ndimage import zoom

def ajustar_atlas_para_imagem(atlas, target_shape):
    """
    Ajusta o shape do atlas para corresponder ao shape da imagem de entrada.

    Parâmetros:
    - atlas: Array numpy com os dados do atlas.
    - target_shape: Tupla com as dimensões desejadas (shape da imagem de entrada).

    Retorna:
    - Atlas redimensionado para o shape desejado.
    """
    # Calcular os fatores de redimensionamento
    zoom_factors = [t / a for t, a in zip(target_shape, atlas.shape)]
    
    # Redimensionar o atlas
    atlas_resized = zoom(atlas, zoom_factors, order=0)  # `order=0` para preservar rótulos
    return atlas_resized


def calcular_volumes_com_atlas(image_path, atlas_path):
    """
    Calcula os volumes de WM, GM e CSF em mm³ com base em um atlas de referência.

    Parâmetros:
    - image_path: Caminho para o arquivo da imagem segmentada (.nii ou .nii.gz).
    - atlas_path: Caminho para o arquivo do atlas de referência (.nii ou .nii.gz).

    Retorna:
    - Um dicionário com os volumes calculados para WM, GM e CSF em mm³.
    """
    # Carregar a imagem segmentada
    img = nib.load(image_path)
    data = img.get_fdata()

    # Carregar o atlas
    atlas = nib.load(atlas_path)
    atlas_data = atlas.get_fdata()

    # Verificar compatibilidade entre a imagem e o atlas
    if atlas_data.shape != data.shape:
        atlas_data = ajustar_atlas_para_imagem(atlas_data, data.shape)

    # Obter as dimensões do voxel
    voxel_dims = img.header.get_zooms()[:3]
    voxel_volume = np.prod(voxel_dims)  # Volume de um voxel em mm³

    # Identificar rótulos de WM, GM e CSF no atlas
    wm_labels = [1]  # Substitua pelos rótulos reais do atlas para WM
    #gm_labels = [2]  # Substitua pelos rótulos reais do atlas para GM
    #csf_labels = [3]  # Substitua pelos rótulos reais do atlas para CSF

    # Máscaras para WM, GM e CSF
    wm_mask = np.isin(atlas_data, wm_labels)
    #gm_mask = np.isin(atlas_data, gm_labels)
    #csf_mask = np.isin(atlas_data, csf_labels)

    # Calcular volumes
    wm_volume = np.sum(data[wm_mask]) * voxel_volume
    #gm_volume = np.sum(data[gm_mask]) * voxel_volume
    #csf_volume = np.sum(data[csf_mask]) * voxel_volume

    return {
        "WM (mm³)": wm_volume,
        #"GM (mm³)": gm_volume,
        #"CSF (mm³)": csf_volume,
        #"Voxel Dimensions (mm)": voxel_dims,
    }

# Exemplo de uso
if __name__ == "__main__":
    # Substitua pelos caminhos reais para a imagem e o atlas
    image_path = f"work1/clinical_images/IXI012-HH-1211-T1.nii.gz"
    atlas_path = f"work1/atlas/mni_wm.nii.gz"    
    resultados = calcular_volumes_com_atlas(image_path, atlas_path)
    print("Resultados:")
    for key, value in resultados.items():
        print(f"{key}: {value}")


Resultados:
WM (mm³): 404031.08251610334


In [ ]:
print(atlas_path)
img = nib.load(atlas_path)
print(img)